<a href="https://colab.research.google.com/github/Felgaba/HSE_DS_MasterTrack/blob/main/Structured_Streaming_Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q findspark
! pip install -q pyspark

     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 50.8 MB/s 


In [ ]:
import findspark
findspark.init()

import pyspark

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
input_stream_dir = "/stream/structured/"
!mkdir -p $input_stream_dir

In [ ]:
from pyspark.sql import (
    types,
    functions,
)

schema = (
    types.StructType()
    .add("rank", "integer")
    .add("site", "string")
)

spark.readStream.schema(schema).csv(input_stream_dir)

DataFrame[rank: int, site: string]

In [ ]:
input_stream = spark.readStream.schema(schema).csv(input_stream_dir)

In [ ]:
input_stream.writeStream.format("memory").queryName("input").trigger(processingTime="5 seconds").start()

In [ ]:
!shuf /content/drive/MyDrive/Programming/m5-forecasting-accuracy/data/top-1m.csv | head -1000 > `tempfile -d $input_stream_dir`

In [ ]:
spark.sql("SELECT * FROM input").show()

+------+--------------------+
|  rank|                site|
+------+--------------------+
|396128|        artwolfe.com|
|167564|           loves.com|
|188503|         midiinc.com|
| 28367|downforeveryoneor...|
|918875|    sarkarinotes.com|
|995576|      kalittaair.com|
|158285|   usenet-driver.top|
|245425|         i-popka.net|
|607034|          namtia.com|
|447926|         petedge.com|
|   808|         doorblog.jp|
|645561|           blasty.co|
|412815|confederationpays...|
| 48404|motorcyclespecs.c...|
|966704|         dusupay.com|
|430323|             kchr.ru|
|898407|      madresenama.ir|
|682614|  shop-collect.co.il|
|888976|       tweetsave.com|
|379309|sozdaniegrupp.blo...|
+------+--------------------+
only showing top 20 rows



In [ ]:
!head /content/drive/MyDrive/Programming/m5-forecasting-accuracy/data/ips

google.com,172.217.17.110
youtube.com,216.58.211.110
facebook.com,157.240.20.35
baidu.com,123.125.114.144
baidu.com,220.181.38.148
wikipedia.org,91.198.174.192
qq.com,111.161.64.48
qq.com,111.161.64.40
taobao.com,140.205.94.189
taobao.com,140.205.220.96


In [ ]:
ip_schema = (
    types.StructType()
    .add("site", "string")
    .add("ip", "string")
)

ip_df = spark.read.schema(ip_schema).csv("/content/drive/MyDrive/Programming/m5-forecasting-accuracy/data/ips")

In [ ]:
ip_df.show()

+-------------+---------------+
|         site|             ip|
+-------------+---------------+
|   google.com| 172.217.17.110|
|  youtube.com| 216.58.211.110|
| facebook.com|  157.240.20.35|
|    baidu.com|123.125.114.144|
|    baidu.com| 220.181.38.148|
|wikipedia.org| 91.198.174.192|
|       qq.com|  111.161.64.48|
|       qq.com|  111.161.64.40|
|   taobao.com| 140.205.94.189|
|   taobao.com| 140.205.220.96|
|    yahoo.com|    72.30.35.10|
|    yahoo.com|   98.137.246.7|
|    yahoo.com|   98.137.246.8|
|    yahoo.com|     72.30.35.9|
|    yahoo.com| 98.138.219.231|
|    yahoo.com| 98.138.219.232|
|    tmall.com| 140.205.94.193|
|    tmall.com| 140.205.130.99|
|   amazon.com|  176.32.98.166|
|   amazon.com|205.251.242.103|
+-------------+---------------+
only showing top 20 rows



In [ ]:
annotated_df = input_stream.join(
    ip_df, 
    on=(input_stream.site == ip_df.site)
)

In [ ]:
annotated_df.writeStream.format("memory").queryName("annotated_1").trigger(processingTime="5 seconds").start()

In [ ]:
spark.sql("SELECT * FROM annotated_1").show()

+-----+--------------------+--------------------+---------------+
| rank|                site|                site|             ip|
+-----+--------------------+--------------------+---------------+
| 3348|       stripchat.com|       stripchat.com|   88.208.29.18|
| 4046| futura-sciences.com| futura-sciences.com|   5.135.141.70|
| 4412|         commank.pro|         commank.pro|   88.85.94.236|
| 5017|          bunshun.jp|          bunshun.jp|202.238.151.148|
| 5609|        optus.com.au|        optus.com.au|  104.74.53.215|
| 5609|        optus.com.au|        optus.com.au|104.116.136.183|
| 5609|        optus.com.au|        optus.com.au| 23.212.208.171|
| 6892|         gbatemp.net|         gbatemp.net|   23.96.82.247|
| 7344|         studocu.com|         studocu.com|   52.18.170.98|
| 7344|         studocu.com|         studocu.com|  34.253.165.49|
| 7344|         studocu.com|         studocu.com|  52.209.177.31|
| 8307|          zocdoc.com|          zocdoc.com|    45.60.31.63|
| 8307|   

In [ ]:
spark.sql("SELECT count(1) FROM annotated_1").show()

+--------+
|count(1)|
+--------+
|     145|
+--------+



In [ ]:
!shuf top-1m.csv | head -10000 > `tempfile -d $input_stream_dir`

shuf: top-1m.csv: No such file or directory


In [ ]:
spark.sql("SELECT count(1) FROM annotated_1").show()

+--------+
|count(1)|
+--------+
|     145|
+--------+



In [ ]:
spark.sql("SELECT * FROM annotated_1").show()

+-----+--------------------+--------------------+---------------+
| rank|                site|                site|             ip|
+-----+--------------------+--------------------+---------------+
| 3348|       stripchat.com|       stripchat.com|   88.208.29.18|
| 4046| futura-sciences.com| futura-sciences.com|   5.135.141.70|
| 4412|         commank.pro|         commank.pro|   88.85.94.236|
| 5017|          bunshun.jp|          bunshun.jp|202.238.151.148|
| 5609|        optus.com.au|        optus.com.au|  104.74.53.215|
| 5609|        optus.com.au|        optus.com.au|104.116.136.183|
| 5609|        optus.com.au|        optus.com.au| 23.212.208.171|
| 6892|         gbatemp.net|         gbatemp.net|   23.96.82.247|
| 7344|         studocu.com|         studocu.com|   52.18.170.98|
| 7344|         studocu.com|         studocu.com|  34.253.165.49|
| 7344|         studocu.com|         studocu.com|  52.209.177.31|
| 8307|          zocdoc.com|          zocdoc.com|    45.60.31.63|
| 8307|   

In [ ]:
ip_stream_dir = "/stream/ips/"

!mkdir -p $ip_stream_dir

In [ ]:
ips_stream = spark.readStream.schema(ip_schema).csv(ip_stream_dir)

In [ ]:
(
    ips_stream
    .join(input_stream, on="site")
    .writeStream
    .format("memory")
    .trigger(processingTime="5 seconds")
    .queryName("stream_stream_2")
    .start()
)

In [ ]:
spark.sql("SELECT * FROM stream_stream_2").show()

+--------------------+--------------+-----+
|                site|            ip| rank|
+--------------------+--------------+-----+
|      triplem.com.au| 202.59.43.101|42798|
|           ibood.com|194.233.193.45|23374|
|       bancor.com.ar|200.45.254.228|27957|
|           mjobs.net|109.199.116.18|62453|
|             mba.com|107.154.102.70|20690|
|buybitcoinworldwi...|104.27.189.130|59462|
|            cutw.pro| 104.31.76.152|14372|
+--------------------+--------------+-----+



In [ ]:
!ls $ip_stream_dir

file0BeiBm  filegx73Pk


In [ ]:
!shuf /content/drive/MyDrive/Programming/m5-forecasting-accuracy/data/ips | head -10000 > `tempfile -d $ip_stream_dir`

In [ ]:
spark.sql("SELECT * FROM stream_stream_2").show()

+--------------------+--------------+-----+
|                site|            ip| rank|
+--------------------+--------------+-----+
|      triplem.com.au| 202.59.43.101|42798|
|           ibood.com|194.233.193.45|23374|
|       bancor.com.ar|200.45.254.228|27957|
|           mjobs.net|109.199.116.18|62453|
|             mba.com|107.154.102.70|20690|
|buybitcoinworldwi...|104.27.189.130|59462|
|            cutw.pro| 104.31.76.152|14372|
+--------------------+--------------+-----+

